In [3]:
import random as rd

Variant = 4
rd.seed(Variant)

Numbers_of_problems = [rd.sample(range(4),1)[0]+1, rd.sample(range(4),1)[0]+1]
Numbers_of_problems

[2, 3]

2. С использованием библиотеки ncurses реализовать случайное блуждание курсора по экрану, курсор не должен уходить за границы экрана, при соударении с границей, он должен менять цвет.

In [ ]:
format ELF64

public _start
extrn initscr
extrn printw
extrn refresh
extrn getch
extrn endwin
extrn exit
extrn stdscr
extrn getmaxx
extrn getmaxy
extrn move
extrn attron
extrn attroff
extrn COLOR_PAIR
extrn start_color
extrn init_pair

section '.data'

section '.bss'
    cursor_x resq 1
    cursor_y resq 1

section '.text' executable

_start:
    call initscr
    call start_color
    mov rdi, 1
    mov rsi, 1
    mov rdx, 7
    call init_pair
    mov rdi, [stdscr]
    call getmaxx
    mov r8, rax
    call getmaxy
    mov r9, rax
    mov rax, r9
    mov rcx, 2
    div rcx
    mov [cursor_y], rax
    mov rax, r8
    mov rcx, 2
    div rcx
    mov [cursor_x], rax

.loop:
    mov rdi, [cursor_y]
    mov rsi, [cursor_x]
    call move
    mov rdi, " "
    call printw
    call refresh
    call getch
    mov rdi, [cursor_y]
    cmp rdi, 0
    jl .change_color
    cmp rdi, r9
    jge .change_color
    mov rsi, [cursor_x]
    cmp rsi, 0
    jl .change_color
    cmp rsi, r8
    jge .change_color
    jmp .loop

.change_color:
    call attron
    mov rdi, COLOR_PAIR
    mov rsi, 1
    call attron
    jmp .loop

end:
    call endwin
    call exit


3. В программе на ассемблере заполнить массив числами 1, 2, 3,... в количестве $N$ (вводится параметром командной строки), создать два дочерних процесса (clone). Один дочерний процесс должен увеличить на единицу все четные числа, другой - уменьшить на единицу все нечетные. По окончанию работы дочерных процессов родительский процесс должен вывести получившийся массив на экран.

In [ ]:
format elf64

public _start

section '.data' writable
    msg db "Enter the size of array (N): ", 0
    newline db 0xa, 0
    result_msg db "Modified array: ", 0

section '.bss' writable
    n_times resq 1  ; Для хранения размера массива (8 байт)
    array resq 100  ; Для хранения массива (100 элементов, можно увеличить по необходимости)

section '.text' executable

_start:
    ; Печатаем сообщение о вводе размера массива
    mov rsi, msg
    call print_str
    call new_line

    ; Получаем параметр командной строки (размер массива N)
    mov rsi, [rsp+8]    ; Параметры командной строки начинаются с rsi (первый параметр — путь к программе)
    call atoi           ; Преобразуем параметр в число и сохраняем в n_times
    mov [n_times], rax

    ; Заполняем массив числами от 1 до N
    mov rdi, [n_times]  ; Количество элементов в массиве
    mov rbx, 1           ; Начальное значение для заполнения массива

fill_array:
    mov [array + rbx - 1], rbx
    inc rbx
    dec rdi
    jnz fill_array

    ; Создаем первый дочерний процесс для увеличения четных чисел
    mov rax, 56         ; Системный вызов clone
    mov rdi, 0          ; Для простоты не используем флаги
    mov rsi, child1
    syscall

    ; Создаем второй дочерний процесс для уменьшения нечетных чисел
    mov rax, 56         ; Системный вызов clone
    mov rdi, 0          ; Для простоты не используем флаги
    mov rsi, child2
    syscall

    ; Ожидаем завершения дочерних процессов
    mov rax, 60         ; Системный вызов exit
    xor rdi, rdi        ; Завершаем работу родительского процесса
    syscall

child1:
    ; Первый дочерний процесс: увеличивает все четные числа
    mov rbx, 0          ; Индекс в массиве
    mov rcx, [n_times]  ; Количество элементов массива

child1_loop:
    mov rdx, [array + rbx] ; Получаем число из массива
    test rdx, 1         ; Проверяем, четное ли число
    jnz child1_skip     ; Если нечетное, пропускаем
    inc rdx             ; Увеличиваем на 1, если четное
    mov [array + rbx], rdx ; Сохраняем обратно в массив
child1_skip:
    inc rbx
    dec rcx
    jnz child1_loop
    ret

child2:
    ; Второй дочерний процесс: уменьшает все нечетные числа
    mov rbx, 0          ; Индекс в массиве
    mov rcx, [n_times]  ; Количество элементов массива

child2_loop:
    mov rdx, [array + rbx] ; Получаем число из массива
    test rdx, 1         ; Проверяем, нечетное ли число
    jz child2_skip      ; Если четное, пропускаем
    dec rdx             ; Уменьшаем на 1, если нечетное
    mov [array + rbx], rdx ; Сохраняем обратно в массив
child2_skip:
    inc rbx
    dec rcx
    jnz child2_loop
    ret

; Функция для вывода строки
print_str:
    mov rax, 0x1        ; Системный вызов write
    mov rdi, 0x1        ; Стандартный вывод
    mov rdx, [rsi-1]    ; Длина строки
    syscall
    ret

; Функция для вывода новой строки
new_line:
    mov rsi, newline
    call print_str
    ret

; Функция для преобразования строки в число (atoi)
atoi:
    xor rax, rax            ; Обнуляем rax (будет числовое значение)
atoi_loop:
    movzx rbx, byte [rsi]   ; Берем символ
    test rbx, rbx            ; Если ноль, конец строки
    jz atoi_done
    sub rbx, '0'             ; Преобразуем в число
    imul rax, rax, 10        ; Умножаем на 10 (сдвиг влево на 1 разряд)
    add rax, rbx             ; Прибавляем текущую цифру
    inc rsi
    jmp atoi_loop
atoi_done:
    ret
